In [1]:
# Instalação de bibliotecas
import sys
from pathlib import Path
import importlib
import os
import zipfile
import gdown
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from time import perf_counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enibe os warnings
import warnings
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names"
)


# Caminho relativos
NOTEBOOK_FILE = Path(__vsc_ipynb_file__).resolve()

NOTEBOOKS_PATH = NOTEBOOK_FILE.parent          # notebooks
SEMANA_PATH    = NOTEBOOKS_PATH.parent         # semana
SCRIPTS_PATH   = SEMANA_PATH / "scripts"       # scripts
OUTPUTS_PATH   = SEMANA_PATH / "outputs"       # outputs
PROJECT_PATH   = SEMANA_PATH.parent            # projeto

print("Caminhos relativos \n", 
      "Notebook:", NOTEBOOKS_PATH, "\n", 
      "Semana:", SEMANA_PATH,  "\n",
      "Script", SCRIPTS_PATH, "\n",
      "Outputs", OUTPUTS_PATH)


# Import do script com as funções da equipe
sys.path.insert(0, str(SCRIPTS_PATH))
import flight_delay_pipeline as scr
importlib.reload(scr)
print("Script importado da pasta:", SCRIPTS_PATH)

Caminhos relativos 
 Notebook: C:\Users\helen\OneDrive\Área de Trabalho\OnFlight\Projeto OnFlight\Hackaton\Hackaton\data_science\semana_04\notebooks 
 Semana: C:\Users\helen\OneDrive\Área de Trabalho\OnFlight\Projeto OnFlight\Hackaton\Hackaton\data_science\semana_04 
 Script C:\Users\helen\OneDrive\Área de Trabalho\OnFlight\Projeto OnFlight\Hackaton\Hackaton\data_science\semana_04\scripts 
 Outputs C:\Users\helen\OneDrive\Área de Trabalho\OnFlight\Projeto OnFlight\Hackaton\Hackaton\data_science\semana_04\outputs
Script importado da pasta: C:\Users\helen\OneDrive\Área de Trabalho\OnFlight\Projeto OnFlight\Hackaton\Hackaton\data_science\semana_04\scripts


In [2]:
# Carregar Dados
file_id = "1207psedBKvnS0pJkDITroSzPiWrcz0ag"

# Nome do arquivo que será baixado no Colab
zip_path = "dados_vra.zip"

# Se ainda não existir o zip, baixa do Drive
if not os.path.exists(zip_path):
    url = f"https://drive.google.com/uc?id={file_id}"
    print("Baixando arquivo do Drive...")
    gdown.download(url, zip_path, quiet=False)
else:
    print("Arquivo ZIP já existe, download interrompido.")

# Extrair o conteúdo do zip
extract_folder = "dados_vra"

if not os.path.exists(extract_folder):
    print("Extraindo arquivos do ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_folder)
else:
    print("Pasta de dados já existe.")

Arquivo ZIP já existe, download interrompido.
Pasta de dados já existe.


In [3]:
# Carregamento do Dataset
df = scr.carregar_dataset_base(
    pasta = "./dados_vra/dados_vra",  # ajuste aqui se necessário
    sep=";",
    encoding="latin-1",
    skiprows=1,
    renomear=True,
    converter_datas=True,
)

df.head()

C:\Users\helen\OneDrive\Área de Trabalho\OnFlight\Projeto OnFlight\Hackaton\Hackaton\data_science\semana_04\scripts\flight_delay_pipeline.py:152: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


,empresa_aerea,numero_voo,codigo_autorizacao_di,codigo_tipo_linha,aerodromo_origem,aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,codigo_justificativa
0,TAM,3447,0,N,SBGR,SBFI,2021-01-09 20:25:00,2021-01-09 20:21:00,2021-01-09 22:05:00,2021-01-09 21:56:00,REALIZADO,NaN
1,TAM,3447,0,N,SBGR,SBFI,2021-01-10 20:25:00,2021-01-10 20:24:00,2021-01-10 22:05:00,2021-01-10 21:55:00,REALIZADO,NaN
2,TAM,3447,0,N,SBGR,SBFI,2021-01-11 20:25:00,2021-01-11 20:27:00,2021-01-11 22:05:00,2021-01-11 21:58:00,REALIZADO,NaN
3,TAM,3447,0,N,SBGR,SBFI,2021-01-12 20:25:00,2021-01-12 20:24:00,2021-01-12 22:05:00,2021-01-12 21:58:00,REALIZADO,NaN
4,TAM,3447,0,N,SBGR,SBFI,2021-01-13 20:25:00,2021-01-13 20:16:00,2021-01-13 22:05:00,2021-01-13 21:48:00,REALIZADO,NaN


In [4]:
#summary = scr.eda_viz(df, target=None)

In [5]:
df_flags = scr.criar_flags_qualidade_basicas(df)

df_flags[[
    "flag_partida_prevista_ausente",
    "flag_partida_real_ausente",
    "flag_data_fora_periodo"
]].mean()

flag_partida_prevista_ausente    0.041119
flag_partida_real_ausente        0.042030
flag_data_fora_periodo           0.000000
dtype: float64

In [6]:
df_model = scr.criar_target_atrasado(df_flags, limite_min=15)

df_model[scr.TARGET_COL].value_counts(normalize=True)
df_model.shape

(3644100, 17)

In [7]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3644100 entries, 0 to 3968417
Data columns (total 17 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   empresa_aerea                  object        
 1   numero_voo                     object        
 2   codigo_autorizacao_di          object        
 3   codigo_tipo_linha              object        
 4   aerodromo_origem               object        
 5   aerodromo_destino              object        
 6   partida_prevista               datetime64[ns]
 7   partida_real                   datetime64[ns]
 8   chegada_prevista               datetime64[ns]
 9   chegada_real                   datetime64[ns]
 10  situacao_voo                   object        
 11  codigo_justificativa           float64       
 12  flag_partida_prevista_ausente  bool          
 13  flag_partida_real_ausente      bool          
 14  flag_data_fora_periodo         bool          
 15  atraso_partida_min  

In [8]:
df_model.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
empresa_aerea,3644100,157,AZU,1229715,NaN,NaN,NaN,NaN,NaN,NaN,NaN
numero_voo,3644100,7430,0506,4327,NaN,NaN,NaN,NaN,NaN,NaN,NaN
codigo_autorizacao_di,3644100,9,0,3612381,NaN,NaN,NaN,NaN,NaN,NaN,NaN
codigo_tipo_linha,3644100,5,N,3107397,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aerodromo_origem,3644100,411,SBGR,519609,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aerodromo_destino,3644100,406,SBGR,522235,NaN,NaN,NaN,NaN,NaN,NaN,NaN
partida_prevista,3644100,NaN,NaN,NaN,2023-06-07 10:31:24.035218432,2021-01-01 00:05:00,2022-06-06 18:18:45,2023-06-25 11:55:00,2024-06-30 19:10:00,2025-07-01 07:10:00,NaN
partida_real,3644100,NaN,NaN,NaN,2023-06-07 10:37:23.615904768,2020-12-31 23:51:00,2022-06-06 18:30:00,2023-06-25 11:59:00,2024-06-30 19:13:30,2025-07-01 12:46:00,NaN
chegada_prevista,3644100,NaN,NaN,NaN,2023-06-07 12:58:50.578217216,2021-01-01 02:45:00,2022-06-06 20:25:00,2023-06-25 14:00:00,2024-06-30 21:30:00,2025-07-01 13:45:00,NaN
chegada_real,3644098,NaN,NaN,NaN,2023-06-07 13:02:12.319344384,2021-01-01 02:16:00,2022-06-06 20:34:15,2023-06-25 14:06:00,2024-06-30 21:33:00,2025-07-01 22:05:00,NaN


In [9]:
# Definição dos conjuntos de treino, validação e teste
time_col = "partida_prevista"
df_train, df_val, df_test = scr.criar_split_temporal_train_val_test(
    df_model,
    time_col=time_col,
    train_size=0.7,
    val_size=0.1
)

# Checagem dos conjuntos
print("Formato do conjunto de treino: ", df_train.shape)
print("Periodo dos dados de treino: ", df_train[time_col].min(), "→", df_train[time_col].max())
print("-"*80, "\n")
print("Formato do conjunto de validação: ", df_val.shape)
print("Periodo do dados de valiação: ", df_val[time_col].min(), "→", df_val[time_col].max())
print("-"*80, "\n")
print("Formato do conjunto de teste: ", df_test.shape)
print("Periodo do dados de teste: ", df_test[time_col].min(), "→", df_test[time_col].max())

Formato do conjunto de treino:  (2550870, 17)
Periodo dos dados de treino:  2021-01-01 00:05:00 → 2024-04-16 10:20:00
-------------------------------------------------------------------------------- 

Formato do conjunto de validação:  (364409, 17)
Periodo do dados de valiação:  2024-04-16 10:25:00 → 2024-09-09 08:00:00
-------------------------------------------------------------------------------- 

Formato do conjunto de teste:  (728821, 17)
Periodo do dados de teste:  2024-09-09 08:00:00 → 2025-07-01 07:10:00


In [10]:
# Definição das colunas que irão para o modelo
cfg = scr.FeatureConfig(
    numeric_features=[
        # criadas pelo função DatasFeaturesTransformer
        "hora_dia",
        "dia_semana",
        "mes_ano",
        "fim_de_semana",
        "alta_temporada",
        "hora_sin",
        "hora_cos",
        "is_hub",

        # criadas pelo pela função MediaAtrasoTransformer
        "media_atraso_empresa",
        "media_atraso_origem",
        "media_atraso_destino",
    ],
    categorical_features=[
        # originais
        #"empresa_aerea",
        "codigo_tipo_linha",
        #"aerodromo_origem",
        #"aerodromo_destino",
        #"situacao_voo",                >>> DATA LACKAGE - ACONTECE APÓS A PARTIDA DO VOO

        # criada pela função DatasFeaturesTransformer
        "periodo_dia",
    ],
)

In [11]:
cols_removida_modelagem = [
    # vazamento de informação
    "partida_real", "chegada_real", 
    "situacao_voo", "codigo_justificativa",
    "atraso_partida_min", "flag_partida_real_ausente", 
    
    # informação de identificação
    "numero_voo", "codigo_autorizacao_di",

    # categóricas de alta cardinalidade 
    "aerodromo_origem", "aerodromo_destino", "empresa_aerea"
]

In [12]:
# Construção do conjunto de dados de entrada para a checagem
X_chk = df_train.drop(columns=[scr.TARGET_COL]).copy()

# Execução isolada do Feature Engineering
fe_chk = scr.Pipeline(steps=[
    ("datas", scr.DatasFeaturesTransformer(col_dt="partida_prevista", col_atraso="atraso_partida_min")),
    ("ultimate", scr.UltimateFeatureEngineer()),
    ("medias", scr.MediaAtrasoTransformer(col_atraso="atraso_partida_min")),
    ("drop_leakage", scr.DropColumnsTransformer(cols_removida_modelagem)),
])
Xfe_chk = fe_chk.fit_transform(X_chk)

# Verificação de colunas esperadas vs. colunas geradas
missing = [c for c in (cfg.numeric_features + cfg.categorical_features) if c not in Xfe_chk.columns]
print("Nenhuma coluna está faltando.\n" if not missing else f"Colunas faltando: {missing}\n")

# Verificação pra ver se colunas de vazamento passou para o modelo
for col in cols_removida_modelagem:
    print(col, col in Xfe_chk.columns)


Nenhuma coluna está faltando.

partida_real False
chegada_real False
situacao_voo False
codigo_justificativa False
atraso_partida_min False
flag_partida_real_ausente False
numero_voo False
codigo_autorizacao_di False
aerodromo_origem False
aerodromo_destino False
empresa_aerea False


In [13]:
# Checagem pra ver o numero de features enviadas ao modelo
pre_chk = scr.montar_preprocessador(cfg)
pre_chk.fit(Xfe_chk)

Xproc_chk = pre_chk.transform(Xfe_chk.iloc[:1000])
print("Nº colunas após processamento:", Xproc_chk.shape[1])

try:
    fn = pre_chk.get_feature_names_out()
    print(f"Total de features finais enviadas para o modelo: {len(fn)}")
    print("Exemplo:", fn[:500])
except Exception:
    print("Preprocessador não expõe nomes das features.")

Nº colunas após processamento: 20
Total de features finais enviadas para o modelo: 20
Exemplo: ['num__hora_dia' 'num__dia_semana' 'num__mes_ano' 'num__fim_de_semana'
 'num__alta_temporada' 'num__hora_sin' 'num__hora_cos' 'num__is_hub'
 'num__media_atraso_empresa' 'num__media_atraso_origem'
 'num__media_atraso_destino' 'cat__codigo_tipo_linha_C'
 'cat__codigo_tipo_linha_G' 'cat__codigo_tipo_linha_I'
 'cat__codigo_tipo_linha_N' 'cat__codigo_tipo_linha_infrequent_sklearn'
 'cat__periodo_dia_Madrugada' 'cat__periodo_dia_Manha'
 'cat__periodo_dia_Noite' 'cat__periodo_dia_Tarde']


In [14]:
# Converte a varial alvo para formato numérico (exigido pelo SMOTE)
y_train = df_train[scr.TARGET_COL].astype(int)

# Exibe a distribuição das classes
class_counts = y_train.value_counts()
print("Distribuição das classes:")
print(class_counts)

# Exibe o número de amostras da classe minoritária
n_minoritario = class_counts.min()
print(f"\nAmostras na classe minoritária: {n_minoritario}")

# Definição segura do k_neighbors para o SMOTE
# Regra:
# - k_neighbors < n_minoritario
# - valor padrão recomendado pela documentação = 5
k_neighbors_smote = min(5, n_minoritario - 1)
k_neighbors_smote = max(1, k_neighbors_smote)

print(f"k_neighbors recomendado para SMOTE: {k_neighbors_smote}")

Distribuição das classes:
atrasado
0    2151215
1     399655
Name: count, dtype: int64

Amostras na classe minoritária: 399655
k_neighbors recomendado para SMOTE: 5


In [15]:
# Treinamento com os dados de 2021 e 2022 e avaliação com os dados de 2023
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=2000, solver="saga", class_weight=None, tol=1e-3),
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=10, class_weight=None, random_state=42, n_jobs=1),
    "XGBoost": XGBClassifier(tree_method="hist", n_jobs=1, random_state=42, scale_pos_weight=1, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(class_weight=None, verbose=-1, n_jobs=1)
}

resultados_val = []
pipes_val = {}

for nome, modelo in modelos.items():
    out = scr.treinar_classificador(
        df_train=df_train,              # Periodo dos dados de treino: 2021-01-01 00:05:00 → 2024-04-16 10:20:00
        df_test=df_val,                 # Periodo dos dados de avaliação: 2024-04-16 10:25:00 → 2024-09-09 08:00:00
        cfg=cfg,
        model=modelo,
        drop=cols_removida_modelagem,
        target=scr.TARGET_COL,
        use_smote=True,
        smote_k_neighbors= 5
    )

    met = scr.extrair_metricas(out, positive_label="1")
    resultados_val.append({"Modelo": nome, **met})
    pipes_val[nome] = out["pipeline"]

df_val_resultados = (
    pd.DataFrame(resultados_val)
      .sort_values(by="f1_pos", ascending=False)
      .reset_index(drop=True)
)

df_val_resultados

--------------------------------------------------------------------------- 
 LogisticRegression(max_iter=2000, solver='saga', tol=0.001) 
 ---------------------------------------------------------------------------
Fit OK. Pipeline fitted.
Faltando para o preprocessador: []
Check de fitagem OK.
--------------------------------------------------------------------------- 
 RandomForestClassifier(max_depth=10, n_estimators=200, n_jobs=1,
                       random_state=42) 
 ---------------------------------------------------------------------------
Fit OK. Pipeline fitted.
Faltando para o preprocessador: []
Check de fitagem OK.
--------------------------------------------------------------------------- 
 XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              featur

,Modelo,accuracy,precision_pos,recall_pos,f1_pos,support_pos,f1_macro,f1_weighted,roc_auc,cm
0,Random Forest,0.714335,0.245235,0.449805,0.317415,53810.0,0.568393,0.745250,0.652107,"[[236106, 74493], [29606, 24204]]"
1,LightGBM,0.808155,0.333506,0.299665,0.315681,53810.0,0.602061,0.803864,0.669135,"[[278374, 32225], [37685, 16125]]"
2,XGBoost,0.814360,0.344642,0.285263,0.312154,53810.0,0.602427,0.806975,0.671984,"[[281410, 29189], [38460, 15350]]"
3,Logistic Regression,0.591058,0.200640,0.592957,0.299827,53810.0,0.505507,0.650445,0.636012,"[[183480, 127119], [21903, 31907]]"


In [16]:
# Escolha do melhor modelo de validação
melhor_nome = df_val_resultados.loc[0, "Modelo"]
print("Melhor no VAL:", melhor_nome)

Melhor no VAL: Random Forest


In [17]:
# Treinamento com os dados de 2021 a 2023 e teste com os dados de 2024
melhor_modelo = modelos[melhor_nome]

df_trainval = pd.concat([df_train, df_val], axis=0).sort_values("partida_prevista")     # concatena o periodo de 2021-01-01 00:05:00 a 2024-09-09 08:00:00

out_test = scr.treinar_classificador(
    df_train=df_trainval,               # Periodo dos dados de treino: 2021-01-01 00:05:00 a 2024-09-09 08:00:00
    df_test=df_test,                    # Periodo dos dados de avaliação:  2024-09-09 08:00:00 → 2025-07-01 07:10:00
    cfg=cfg,
    model=melhor_modelo,
    drop=cols_removida_modelagem,
    target=scr.TARGET_COL,
    use_smote=False
)

scr.extrair_metricas(out_test, positive_label="1")

--------------------------------------------------------------------------- 
 RandomForestClassifier(max_depth=10, n_estimators=200, n_jobs=1,
                       random_state=42) 
 ---------------------------------------------------------------------------
Fit OK. Pipeline fitted.
Faltando para o preprocessador: []
Check de fitagem OK.


{'accuracy': 0.8257583137697734,
 'precision_pos': 0.7068351726593094,
 'recall_pos': 0.030707961180064185,
 'f1_pos': 0.058858841054449246,
 'support_pos': 129315.0,
 'f1_macro': 0.4814252865869551,
 'f1_weighted': 0.7540394670117432,
 'roc_auc': 0.6660718519671518,
 'cm': array([[597859,   1647],
        [125344,   3971]])}

In [ ]:
# Treinamento com validação pelo TimeSeriesSplit
X_trainval = df_trainval.drop(columns=[scr.TARGET_COL]).copy()
y_trainval = df_trainval[scr.TARGET_COL].astype(int).copy()

X_test = df_test.drop(columns=[scr.TARGET_COL]).copy()
y_test = df_test[scr.TARGET_COL].astype(int).copy()

# Montagem do pre-processamento manual para controle do GridSearch
fe = Pipeline(steps=[
    ("datas", scr.DatasFeaturesTransformer(col_dt="partida_prevista", col_atraso="atraso_partida_min")),
    ("ultimate", scr.UltimateFeatureEngineer()),
    ("medias", scr.MediaAtrasoTransformer(col_atraso="atraso_partida_min")),
    ("drop_leakage", scr.DropColumnsTransformer(cols_removida_modelagem)),
])

pre = scr.montar_preprocessador(cfg)

pipe = Pipeline(steps=[
    ("fe", fe),
    ("pre", pre),
    ("model", melhor_modelo)
])

tscv = TimeSeriesSplit(n_splits=3)
scorer = make_scorer(f1_score, pos_label=1)
param_grid= {
    "model__n_estimators": [200, 300],
    "model__max_depth": [8, 10],
    "model__min_samples_split": [2, 5],
}
gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scorer,
    cv=tscv,
    n_jobs=1,
    verbose=2
)

gs.fit(X_trainval, y_trainval)
best_pipe = gs.best_estimator_

y_pred = best_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END model__max_depth=8, model__min_samples_split=2, model__n_estimators=200; total time=1397.2min
[CV] END model__max_depth=8, model__min_samples_split=2, model__n_estimators=200; total time=1063.1min


In [ ]:
cols_metricas = ["accuracy", "precision_pos", "recall_pos", "f1_pos", "f1_macro", "f1_weighted", "roc_auc"]

# Ranking baseline
if "df_val_resultados" not in globals():
    raise ValueError("df_val_resultados não existe. Rode o cell que gera df_val_resultados primeiro.")

missing = [c for c in (["Modelo"] + cols_metricas) if c not in df_val_resultados.columns]
if missing:
    raise ValueError(f"df_val_resultados está sem colunas esperadas: {missing}")

df_val_plot = df_val_resultados  

best_name = globals().get(
    "melhor_nome",
    df_val_plot.sort_values("f1_pos", ascending=False).iloc[0]["Modelo"]
)

best_baseline_val = (
    df_val_plot.loc[df_val_plot["Modelo"] == best_name, cols_metricas]
    .iloc[0]
    .to_dict()
)

# Baseline teste (retrain)
if "out_test" not in globals():
    raise ValueError("out_test não existe. Rode o cell de treino final (train+val - test) e salve em out_test.")

best_baseline_test = scr.extrair_metricas(out_test, positive_label="1")
best_baseline_test = {k: best_baseline_test.get(k, np.nan) for k in cols_metricas}

# Tuned teste com GridSearch
if "df_test" not in globals():
    raise ValueError("df_test não existe. É necessário gerar o df_test (split temporal) para avaliar o tuned no teste.")

X_test = df_test.drop(columns=[scr.TARGET_COL])
y_test = df_test[scr.TARGET_COL].astype(int)

best_pipe = globals().get("best_pipe")
gs = globals().get("gs")

pipe_tuned = (
    best_pipe if best_pipe is not None
    else gs.best_estimator_ if (gs is not None and hasattr(gs, "best_estimator_"))
    else None
)
if pipe_tuned is None:
    raise ValueError(
        "Não foi encontrado best_pipe nem gs.best_estimator_. "
        "Rode o GridSearch antes ou passe best_pipe."
    )

y_pred_tuned = pipe_tuned.predict(X_test)

y_proba_tuned = None
if hasattr(pipe_tuned, "predict_proba"):
    try:
        y_proba_tuned = pipe_tuned.predict_proba(X_test)[:, 1]
    except Exception:
        y_proba_tuned = None

# Métricas tuned 
best_tuned_test = {
    "accuracy": float(accuracy_score(y_test, y_pred_tuned)),
    "precision_pos": float(precision_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)),
    "recall_pos": float(recall_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)),
    "f1_pos": float(f1_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)),
    "f1_macro": float(f1_score(y_test, y_pred_tuned, average="macro", zero_division=0)),
    "f1_weighted": float(f1_score(y_test, y_pred_tuned, average="weighted", zero_division=0)),
    "roc_auc": np.nan,
}
if y_proba_tuned is not None:
    try:
        best_tuned_test["roc_auc"] = float(roc_auc_score(y_test, y_proba_tuned))
    except Exception:
        best_tuned_test["roc_auc"] = np.nan

# Tabela de evolução
evolucao = pd.DataFrame([
    {"stage": "Baseline (Validação)", "Modelo": best_name, **best_baseline_val},
    {"stage": "Baseline (Teste, retrain Train+Val)", "Modelo": best_name, **best_baseline_test},
    {"stage": "Tuned (Teste, GridSearch)", "Modelo": best_name, **best_tuned_test},
])

display(evolucao)

# Gráfico comentado
# for m in cols_metricas:
#     fig = plt.figure(figsize=(10, 4))
#     ax = fig.add_subplot(111)
#     ax.plot(evolucao["stage"], evolucao[m], marker="o")
#     ax.set_title(f"Evolução do melhor modelo ({best_name}) — {m}")
#     ax.set_xlabel("")
#     ax.set_ylabel(m)
#     plt.xticks(rotation=20, ha="right")
#     plt.tight_layout()
#     plt.show()


In [ ]:
# Melhor modelo encontrado pelo GridSearch
melhor_modelo = df_val_resultados.iloc[0]["Modelo"]
melhor_pipe = gs.best_estimator_
melhor_params = gs.best_params_
melhor_score_cv = gs.best_score_

print("Melhor Modelo:", melhor_modelo, "\n")
print("Melhor score (CV):", melhor_score_cv, "\n"),
print("Melhor Pipe:", melhor_pipe, "\n")
print("Melhores Parametros:", melhor_params)

In [ ]:
local = explicar_local_unificado(pipe, x_raw=df_um_registro, top_k=8, class_index=1)
df_global = explicar_global_unificado(pipe, top_n=30, importance_type="gain")

In [ ]:
# Constroi o caminho para salvar o modelo
SCRIPTS_PATH = Path(SCRIPTS_PATH)  
model_filename = f"flight_delay_pipeline__best_{melhor_modelo}.pkl"
MODEL_PATH = SCRIPTS_PATH / model_filename

# Salva modelo
scr.salvar_pickle(melhor_pipe, str(MODEL_PATH))
print("Modelo salvo em:", MODEL_PATH)

In [ ]:
explain_filename = f"explain_global.json"
EXPLAIN_GLOBAL_PATH = SCRIPTS_PATH / explain_filename

df_global.to_json(
    EXPLAIN_GLOBAL_PATH,
    orient="records",
    force_ascii=False,
    indent=2
)

print("Explicabilidade global salva em:", EXPLAIN_GLOBAL_PATH)